# GTSF phase: biomass prediction

In this notebook, we are forecasting the weights by finding the closest blender model

### Look at the volumes created with blender

Load blender data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2

import glob
import os
import boto3
import tempfile
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
import tqdm
import pickle

from PIL import Image, ImageDraw




In [ ]:
with open("/root/data/alok/blender_data/volumes_all.json", "r") as f:
    data = json.load(f)

Some plot

In [ ]:
plt.hist(data["volume"])
plt.title("Blender volume histogram")
plt.show()

<h1> Get world keypoint coordinates from GTSF data </h1>

In [ ]:
aws_credentials = json.load(open(os.environ["AWS_CREDENTIALS"]))
s3_client = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")


sql_credentials = json.load(open(os.environ["SQL_CREDENTIALS"]))
sql_engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                           sql_credentials["host"], sql_credentials["port"],
                           sql_credentials["database"]))

Session = sessionmaker(bind=sql_engine)
session = Session()

Base = automap_base()
Base.prepare(sql_engine, reflect=True)
Enclosure = Base.classes.enclosures
Calibration = Base.classes.calibrations
GtsfDataCollection = Base.classes.gtsf_data_collections
StereoFramePair = Base.classes.stereo_frame_pairs


In [ ]:
challenge_data = []
body_parts = ['ADIPOSE_FIN', 'ANAL_FIN', 'TAIL_NOTCH', 'PECTORAL_FIN', 'PELVIC_FIN', 'UPPER_LIP', 'EYE', 'DORSAL_FIN']
for idx, coord in enumerate(data['coordinates']):
    obj = {bp: [1e-2*x for x in coord[bp]] for bp in body_parts}
    obj['biomass'] = 1.88*data['volume'][idx]
    challenge_data.append(obj)

In [ ]:
print(json.dumps(challenge_data[:10], indent=4, sort_keys=True))

<h1> Utility functions for world keypoint normalization </h1>

In [ ]:
def generate_rotation_matrix(u_base, v):
    u = v / np.linalg.norm(v)
    n = np.cross(u_base, u)
    n = n / np.linalg.norm(n)
    theta = -np.arccos(np.dot(u, u_base))

    R = np.array([[
        np.cos(theta) + n[0]**2*(1-np.cos(theta)), 
        n[0]*n[1]*(1-np.cos(theta)) - n[2]*np.sin(theta),
        n[0]*n[2]*(1-np.cos(theta)) + n[1]*np.sin(theta)
    ], [
        n[1]*n[0]*(1-np.cos(theta)) + n[2]*np.sin(theta),
        np.cos(theta) + n[1]**2*(1-np.cos(theta)),
        n[1]*n[2]*(1-np.cos(theta)) - n[0]*np.sin(theta),
    ], [
        n[2]*n[0]*(1-np.cos(theta)) - n[1]*np.sin(theta),
        n[2]*n[1]*(1-np.cos(theta)) + n[0]*np.sin(theta),
        np.cos(theta) + n[2]**2*(1-np.cos(theta))
    ]])
    
    return R

In [ ]:
def normalize_world_keypoints(wkps):
    body_parts = wkps.keys()
    
    # translate keypoints such that tail notch is at origin
    translated_wkps = {bp: wkps[bp] - wkps['TAIL_NOTCH'] for bp in body_parts}
    
    # perform first rotation
    u_base=np.array([1, 0, 0])
    v = translated_wkps['UPPER_LIP']
    R = generate_rotation_matrix(u_base, v)
    norm_wkps_intermediate = {bp: np.dot(R, translated_wkps[bp].T).T for bp in body_parts}
    
    # perform second rotation
    u_base = np.array([0, 0, 1])
#     k = np.array([norm_wkps_intermediate['DORSAL_FIN'][0], 
#                   (norm_wkps_intermediate['DORSAL_FIN'][1] + norm_wkps_intermediate['ADIPOSE_FIN'][1])/2.0,
#                   norm_wkps_intermediate['DORSAL_FIN'][2]])
    
    k = norm_wkps_intermediate['ANAL_FIN']
    v = k - np.array([k[0], 0, 0])
    R = generate_rotation_matrix(u_base, v)
    norm_wkps = {bp: np.dot(R, norm_wkps_intermediate[bp].T).T for bp in body_parts}
    
    return norm_wkps
    


<h1> Utility Method: World Keypoint Calculation </h1>

In [ ]:
# DEFINE OPTICAL PROPERTIES

# all distance are in meters
FOCAL_LENGTH = 0.00843663
BASELINE = 0.128096
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH_PIXEL = FOCAL_LENGTH / PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01035
PIXEL_COUNT_WIDTH = 4096
PIXEL_COUNT_HEIGHT = 3000

def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = PIXEL_COUNT_WIDTH / 2.0  
    image_center_y = PIXEL_COUNT_HEIGHT / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / PIXEL_COUNT_WIDTH)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / PIXEL_COUNT_HEIGHT)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])

def depth_from_disp(disp):
    """ calculate the depth of the point based on the disparity value """
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth

def disp_from_depth(depth):
    disp = FOCAL_LENGTH_PIXEL * BASELINE / depth
    return disp


<h1> Load canonical Blender model </h1>

In [ ]:
def generate_paraboloid_fn(reg):
    c = reg.coef_
    i = reg.intercept_
    
    def paraboloid_fn(x, z):
        return c[0]*x**2 + c[1]*z**2 + c[2]*x*z + c[3]*x + c[4]*z + i
    
    return paraboloid_fn



In [ ]:
blender_model_json = json.load(open('./single.json'))
blender_model = {bp: 1e-2*np.array(blender_model_json['coordinates'][0][bp]) for bp in blender_model_json['coordinates'][0].keys()} 
norm_canonical_wkps = normalize_world_keypoints(blender_model)

# find best fit paraboloid for lateral keypoints
lateral_wkps = norm_canonical_wkps['BODY']
A = np.empty([lateral_wkps.shape[0], 5])
A[:, 0] = lateral_wkps[:, 0]**2
A[:, 1] = lateral_wkps[:, 2]**2
A[:, 2] = lateral_wkps[:, 0] * lateral_wkps[:, 2]
A[:, 3] = lateral_wkps[:, 0]
A[:, 4] = lateral_wkps[:, 2]

b = lateral_wkps[:, 1]


reg = LinearRegression().fit(A, b)
paraboloid_fn = generate_paraboloid_fn(reg)

canonical_volume = blender_model_json['volume'][0]


<h1> Generate accuracy metrics on GTSF data </h1>

In [ ]:
body_parts = [
    'TAIL_NOTCH',
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'DORSAL_FIN',
    'UPPER_LIP',
    'EYE'
]

In [ ]:
def generate_lateral_keypoints(left_keypoints, right_keypoints, world_keypoints, 
                               bp_1='UPPER_LIP', bp_2='TAIL_NOTCH', left_window_size=100, 
                               min_breadth=0.04, max_breadth=0.2):
    left_extrap_kp = (0.5 * left_keypoints[bp_1] + 0.5 * left_keypoints[bp_2]).astype('int64')
    bp_1_depth = world_keypoints[bp_1][1]
    bp_2_depth = world_keypoints[bp_2][1]

    # need to determine lower and upper bounds here in a data driven fashion from GTSF data
    # hardcoded values used here
    extrap_kp_max_depth = (bp_1_depth + bp_2_depth) / 2.0 - min_breadth / 2.0
    extrap_kp_min_depth = (bp_1_depth + bp_2_depth) / 2.0 - max_breadth / 2.0

    # Compute the feature descriptor for the extrapolated keypoint in the left image
    extrap_kp_min_disp = disp_from_depth(extrap_kp_max_depth)
    extrap_kp_max_disp = disp_from_depth(extrap_kp_min_depth)
    
    left_box = left_image[left_extrap_kp[1]-left_window_size//2:left_extrap_kp[1]+left_window_size//2, 
                          left_extrap_kp[0]-left_window_size//2:left_extrap_kp[0]+left_window_size//2]
    right_box = right_image[left_extrap_kp[1]-left_window_size//2:left_extrap_kp[1]+left_window_size//2,
                            left_extrap_kp[0]-int(extrap_kp_max_disp)-left_window_size//2:left_extrap_kp[0]-int(extrap_kp_min_disp)+left_window_size//2]

    
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(left_box,None)
    kp2, des2 = orb.detectAndCompute(right_box,None)
    
    # get top five matches
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1,des2)
    matches = sorted(matches, key = lambda x:x.distance)[:5]
    
    # get world coordinates of lateral keypoints
    lateral_wkps = []
    for match in matches[:5]:
        
        lateral_left_coordinates = np.array(kp1[match.queryIdx].pt).astype(int)
        lateral_left_coordinates[0] += left_extrap_kp[0]-left_window_size//2
        lateral_left_coordinates[1] += left_extrap_kp[1]-left_window_size//2
        
        lateral_right_coordinates = np.array(kp2[match.trainIdx].pt).astype(int)
        lateral_right_coordinates[0] += left_extrap_kp[0]-int(extrap_kp_max_disp)-left_window_size//2
        lateral_right_coordinates[1] += left_extrap_kp[1]-left_window_size//2
        
        disp = abs(lateral_left_coordinates[0] - lateral_right_coordinates[0])
        depth = depth_from_disp(disp)
        lateral_wkp = convert_to_world_point(lateral_left_coordinates[0], lateral_left_coordinates[1], depth)
        lateral_wkps.append(lateral_wkp)
        
    return np.array(lateral_wkps)


In [ ]:
sfps = session.query(StereoFramePair).all()

In [ ]:

left_image_f = 'left_image.jpg'
right_image_f = 'right_image.jpg'

world_keypoints_dict = {}
for row in tqdm.tqdm(sfps):
        
    # download left and right images
    left_image_s3_key, right_image_s3_key, s3_bucket = row.left_image_s3_key, row.right_image_s3_key, row.image_s3_bucket
    s3_client.download_file(s3_bucket, left_image_s3_key, left_image_f)
    s3_client.download_file(s3_bucket, right_image_s3_key, right_image_f)
    
    left_image = cv2.imread(left_image_f)
    right_image = cv2.imread(right_image_f)
    
    # get left, right, and world keypoints
    left_keypoints = json.loads(row.left_image_keypoint_coordinates)
    right_keypoints = json.loads(row.right_image_keypoint_coordinates)
    world_keypoints = json.loads(row.world_keypoint_coordinates)
    
    # convert coordinates from lists to numpy arrays
    left_keypoints = {k: np.array(v) for k, v in left_keypoints.items()}
    right_keypoints = {k: np.array(v) for k, v in right_keypoints.items()}
    world_keypoints = {k: np.array(v) for k, v in world_keypoints.items()}
     
    lateral_wkps = generate_lateral_keypoints(left_keypoints, right_keypoints, world_keypoints)
    world_keypoints['BODY'] = lateral_wkps
    world_keypoints_dict[row.id] = world_keypoints
#     norm_wkps = normalize_world_keypoints(world_keypoints)
        
#     # Determine how to fit canonical Blender model to this GTSF fish
#     x_factor = abs(sum([norm_canonical_wkps[bp][0]*norm_wkps[bp][0] for bp in body_parts]) / \
#                sum([norm_canonical_wkps[bp][0]**2 for bp in body_parts]))
    
#     z_factor = abs(sum([norm_canonical_wkps[bp][2]*norm_wkps[bp][2] for bp in body_parts]) / \
#                sum([norm_canonical_wkps[bp][2]**2 for bp in body_parts]))
    
#     y_factor =  sum([coordinate[1]*paraboloid_fn(coordinate[0], coordinate[2]) for coordinate in norm_wkps['BODY']]) / \
#                 sum([paraboloid_fn(coordinate[0], coordinate[2])**2 for coordinate in norm_wkps['BODY']])
    
#     predicted_volume = canonical_volume*abs(x_factor)*abs(y_factor)*abs(z_factor)
#     predicted_volumes.append(predicted_volume)
#     gt_biomass.append(ground_truth_metadata['data']['weight'])
#     print(ground_truth_metadata['data']['breath'], y_factor)
#     breadths.append(ground_truth_metadata['data']['breath'])
#     y_factors.append(y_factor)



In [ ]:
pickle.dump(world_keypoints_dict, open('world_keypoints_dict.pkl', 'wb'))

In [ ]:
world_keypoints_dict = pickle.load(open('world_keypoints_dict.pkl', 'rb'))

In [ ]:
results_df = pd.DataFrame()
for idx, row in enumerate(sfps):
    if idx % 10 == 0:
        print(idx)
    # get fish_id and ground truth metadata
    gtsf_data_collection_id = row.gtsf_data_collection_id
    gtsf_data_collection = session.query(GtsfDataCollection).get(gtsf_data_collection_id)
    ground_truth_metadata = json.loads(gtsf_data_collection.ground_truth_metadata)
    if ground_truth_metadata['data'].get('species') != 'salmon':
        continue
    
    world_keypoints = world_keypoints_dict[row.id]
    try:
        norm_wkps = normalize_world_keypoints(world_keypoints)

        # Determine how to fit canonical Blender model to this GTSF fish
        x_factor = abs(sum([norm_canonical_wkps[bp][0]*norm_wkps[bp][0] for bp in body_parts]) / \
                   sum([norm_canonical_wkps[bp][0]**2 for bp in body_parts]))

        z_factor = abs(sum([norm_canonical_wkps[bp][2]*norm_wkps[bp][2] for bp in body_parts]) / \
                   sum([norm_canonical_wkps[bp][2]**2 for bp in body_parts]))

        y_factor =  sum([coordinate[1]*paraboloid_fn(coordinate[0] / x_factor, coordinate[2] / z_factor) for coordinate in norm_wkps['BODY']]) / \
                    sum([paraboloid_fn(coordinate[0] / x_factor, coordinate[2] / z_factor)**2 for coordinate in norm_wkps['BODY']])


        # get deviation
        deviation = \
        x_factor * sum([norm_canonical_wkps[bp][0]**2 for bp in body_parts]) - sum([norm_wkps[bp][0]**2 for bp in body_parts]) + \
        z_factor * sum([norm_canonical_wkps[bp][2]**2 for bp in body_parts]) - sum([norm_wkps[bp][2]**2 for bp in body_parts]) + \
        sum([norm_canonical_wkps[bp][1]**2 for bp in body_parts]) - sum([norm_wkps[bp][1]**2 for bp in body_parts])

        predicted_volume = canonical_volume*abs(x_factor)*abs(z_factor)

        row = {
            'deviation': deviation,
            'predicted_volume': predicted_volume,
            'gt_biomass': ground_truth_metadata['data']['weight'],
            'gtsf_data_collection_id': row.gtsf_data_collection_id
        }

        results_df = results_df.append(row, ignore_index=True)
    except:
        pass



In [ ]:
session.rollback()

In [ ]:
results_df['predictions'] = results_df['predicted_volume']*reg.coef_ + reg.intercept_
results_df['abs_difference'] = (results_df.predictions - results_df.gt_biomass).abs()
results_df['pct_difference'] = (results_df.predictions - results_df.gt_biomass).abs() / results_df.gt_biomass

In [ ]:
results_df.sort_values('pct_difference', ascending=False)

In [ ]:
predicted_volumes = results_df.predicted_volume.values
gt_biomass = results_df.gt_biomass.values
predictions = np.array(predicted_volumes)[:, np.newaxis]
reg = LinearRegression().fit(predictions, gt_biomass)
print(reg.coef_, reg.intercept_)
print("R2 : {}".format(reg.score(predictions, gt_biomass)))
predictions = np.squeeze(predictions)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot([0, 5000], [0, 5000], "--", c="r", linewidth=2)
plt.scatter(gt_biomass, predictions*reg.coef_ + reg.intercept_)
plt.xlabel("Ground truth weight")
plt.ylabel("Predicted weight")
plt.axis("scaled")
plt.show()

In [ ]:
sfp = session.query(StereoFramePair).filter(StereoFramePair.gtsf_data_collection_id == 311).all()[0]

# download left and right images
left_image_f = 'left_image.jpg'
right_image_f = 'right_image.jpg'

left_image_s3_key, right_image_s3_key, s3_bucket = sfp.left_image_s3_key, sfp.right_image_s3_key, sfp.image_s3_bucket
s3_client.download_file(s3_bucket, left_image_s3_key, left_image_f)
s3_client.download_file(s3_bucket, right_image_s3_key, right_image_f)

left_image = cv2.imread(left_image_f)
right_image = cv2.imread(right_image_f)

left_keypoints = json.loads(sfp.left_image_keypoint_coordinates)
right_keypoints = json.loads(sfp.right_image_keypoint_coordinates)

im = Image.fromarray(left_image)
draw = ImageDraw.Draw(im)
r = 5
for bp, kp in left_keypoints.items():
    draw.ellipse((kp[0]-r, kp[1]-r, kp[0]+r, kp[1]+r), fill=(255,0,0,255))
    draw.text((kp[0], kp[1]), bp)
im

In [ ]:
im = Image.fromarray(right_image)
draw = ImageDraw.Draw(im)
r = 5
for bp, kp in right_keypoints.items():
    draw.ellipse((kp[0]-r, kp[1]-r, kp[0]+r, kp[1]+r), fill=(255,0,0,255))
    draw.text((kp[0], kp[1]), bp)
im

In [ ]:
sum([paraboloid_fn(coordinate[0], coordinate[2])**2 for coordinate in norm_wkps['BODY']])

In [ ]:
sum([coordinate[1]*paraboloid_fn(coordinate[0], coordinate[2]) for coordinate in norm_wkps['BODY']])

In [ ]:
world_keypoints

In [ ]:
norm_wkps

In [ ]:
gtsf_data_collection_id

In [ ]:
def euclidean_distance(p1, p2):
    return np.linalg.norm(p1 - p2)

In [ ]:
right_keypoints

In [ ]:
world_keypoints

<h1> Get normalized world keyponts of all cached Blender models </h1>

In [ ]:
def euclidean_distance(p1, p2):
    return np.linalg.norm(p1-p2)

In [ ]:
sfps = session.query(StereoFramePair).all()
filtered_sfps = []

# get vector of ground truth biomass
gt_biomass = []
gt_length = []
gt_width = []
gt_breadth = []
gt_kfactor = []
for row in sfps:
    # get ground truth biomass
    gtsf_data_collection_id = row.gtsf_data_collection_id
    gtsf_data_collection = session.query(GtsfDataCollection).get(gtsf_data_collection_id)
    ground_truth_metadata = json.loads(gtsf_data_collection.ground_truth_metadata)
    species = ground_truth_metadata['data'].get('species')
    if species == 'trout':
        continue
    ground_truth_biomass = ground_truth_metadata['data']['weight']
    ground_truth_length = ground_truth_metadata['data']['length']
    ground_truth_width = ground_truth_metadata['data']['width']
    ground_truth_breadth = ground_truth_metadata['data']['breath']
    
    gt_biomass.append(ground_truth_biomass)
    gt_length.append(ground_truth_length)
    gt_width.append(ground_truth_width)
    gt_breadth.append(ground_truth_breadth)
    gt_kfactor.append(ground_truth_biomass / ground_truth_length**3)
    filtered_sfps.append(row)    


In [ ]:
canonical_wkps = {bp: 1e-3*np.array(data['coordinates'][0][bp]) for bp in data['mapping'].keys()}

norm_canonical_wkps = normalize_world_keypoints(canonical_wkps)
canonical_volume = data['volume'][0]

analysis_df = pd.DataFrame()
predicted_volumes = []
y_factors = []
ys = []
for idx, row in enumerate(filtered_sfps):
    # extract and normalize the predicted 3D keypoints
    wkps = json.loads(row.world_keypoint_coordinates)
    wkps = {bp: np.array(wkps[bp]) for bp in wkps.keys()}
    norm_wkps = normalize_world_keypoints(wkps)
    
    x_factor = abs(sum([norm_canonical_wkps[bp][0]*norm_wkps[bp][0]*weight_bp[bp] for bp in data['mapping'].keys()]) / \
               sum([norm_canonical_wkps[bp][0]**2*weight_bp[bp] for bp in data['mapping'].keys()]))
    
    y_factor = abs(sum([norm_canonical_wkps[bp][1]*norm_wkps[bp][1]*weight_bp[bp] for bp in data['mapping'].keys()]) / \
               sum([norm_canonical_wkps[bp][1]**2*weight_bp[bp] for bp in data['mapping'].keys()]))
    
    z_factor = abs(sum([norm_canonical_wkps[bp][2]*norm_wkps[bp][2]*weight_bp[bp] for bp in data['mapping'].keys()]) / \
               sum([norm_canonical_wkps[bp][2]**2*weight_bp[bp] for bp in data['mapping'].keys()]))
    
    volume = canonical_volume * x_factor * z_factor * (1 + (y_factor - 1.0) * 0.12)
    y = norm_wkps['PECTORAL_FIN'][1]-norm_wkps['UPPER_LIP'][1]
    ys.append(y)
    predicted_volumes.append(volume)
    
    


In [ ]:
predictions = np.array(predicted_volumes)[:, np.newaxis]
reg = LinearRegression().fit(predictions, gt_biomass)
print(reg.coef_, reg.intercept_)
print("R2 : {}".format(reg.score(predictions, gt_biomass)))
predictions = np.squeeze(predictions)

In [ ]:
len(gt_biomass), len(predictions)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot([0, 5000], [0, 5000], "--", c="r", linewidth=2)
plt.scatter(gt_biomass, predictions*reg.coef_ + reg.intercept_)
plt.xlabel("Ground truth weight")
plt.ylabel("Predicted weight")
plt.colorbar()
plt.axis("scaled")
plt.show()

In [ ]:
fitted_predictions = predictions*reg.coef_ + reg.intercept_
error = fitted_predictions-gt_biomass
print("Average absolute error: {}".format(np.nanmean(np.abs(error))))
print("Average error: {}".format(np.nanmean(error)))
# error5 = predictions_average-ground_truth
#print("Average absolute error5: {}".format(np.nanmean(np.abs(error5))))
relative_error = ((fitted_predictions-gt_biomass) / gt_biomass)*100
print("Average relative error: {} %".format(np.nanmean(relative_error)))

In [ ]:
def euclidean_distance(p1, p2):
    return np.linalg.norm(p1 - p2)

In [ ]:
for i in range(len(data['coordinates'])):
    canonical_wkps = {bp: 1e-3*np.array(data['coordinates'][i][bp]) for bp in data['mapping'].keys()}
    norm_canonical_wkps = normalize_world_keypoints(canonical_wkps)
    print((norm_canonical_wkps['PECTORAL_FIN'][1]-norm_canonical_wkps['UPPER_LIP'][1]))

In [ ]:
plt.scatter(gt_breadth, ys)
plt.show()